Notebook is also available on my GitHub (https://github.com/amr-ayoub/CarND-LaneLines-P1.git)


*The following libraries are used:

    OpenCV
    Numpy 
    MoviePy
    sklearn (LinearRegression)


Implementation:

1- Reads image.
2- Convert image to gray.
3- Apply gaussian blur
4- Canny edge detection
5- Apply region of interest mask
6- Run Hough transform
7- Determine the lines with the accepted slope and draw them in RED color
8- Draw the longest right and left lines in BLUE after doing extrapolation on them
9- Feed accepted lines plus the extrapolated dominant longest lines to a linear regression to get the main left and right
   lane lines and draw them in WHITE 



In [3]:
    #importing some useful packages
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    import numpy as np
    import cv2
    from sklearn.linear_model import LinearRegression
%matplotlib inline

In [4]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

    
def draw_lines(img, lines, color=[255, 0, 0], thickness=4):
    """
    Draw lines on image:
    
    - Determine the lines with the accepted slope
      and draw them in red color
      
    - Draw the longest right and left lines in blue
      after Extrapolation
      
    - collect the accepted lines plus extrapolated longest lines
      and use them to do
      a linear regression to get the main left and right
      lane lines and draw them in white 
    """
    
    height = img.shape[0]  #height of the image
    width = img.shape[1]   #width
    
    # min and max allowed slopes
    min_slope = (height/3.0)/(width/2.0)
    max_slope = 0.9
    
    # longest_left and right used to store longest left 
    # and right lines to draw them in Green color
    longest_left = [0,0,0,0]
    longest_right = [0,0,0,0]
    
    # X_right and Y_right will be used to collect accepted 
    # lines end points to be used in linear regression
    # to get main lane lines wich is drawn in white color
    X_right = []
    Y_right = []
    
    # X_left and Y_left will be used to collect accepted 
    # lines end points to be used in linear regression
    # to get main lane lines wich is drawn in White color
    X_left = []
    Y_left = []
    
    
    # Draw only lines in the acceptable slopes in red color
    # Collect the end points for the accepted lines to 
    # do linear regression to get the main left and 
    # right lane lines
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            # calculate slope
            slope = ((y2-y1)/(x2-x1))
                       
            # Checking if the line in the accepted range to be drawn
            if ((abs(slope) > min_slope) and ((abs(slope) < max_slope))):
                #Draw lines with accepted slopes in red
                cv2.line(img, (x1, y1), (x2, y2), [255,0,0], 4)
                
                #find the longest line on the right
                if (slope > 0):
                    line_length = math.sqrt((y2-y1)**2 + (x2-x1)**2)
                
                    xx1,yy1,xx2,yy2 = longest_right
                    longest_length = math.sqrt((yy2-yy1)**2 + (xx2-xx1)**2)
                
                    if (line_length > longest_length):
                        longest_right = line[0]
            
                #find the longest line on the left
                if (slope < 0):
                    line_length = math.sqrt((y2-y1)**2 + (x2-x1)**2)
                
                    xx1,yy1,xx2,yy2 = longest_left
                    longest_length = math.sqrt((yy2-yy1)**2 + (xx2-xx1)**2)
                
                    if (line_length > longest_length):
                        longest_left = line[0]
            
                #collecting points from the right lines for linear regression
                if (slope > 0):
                    X_right.append(x1)
                    Y_right.append(y1)
                    X_right.append(x2)
                    Y_right.append(y2)
            
                #collecting points from the left lines for linear regression
                if (slope < 0):
                    X_left.append(x1)
                    Y_left.append(y1)
                    X_left.append(x2)
                    Y_left.append(y2)
    
    
    
    #draw extrapolated longest right lines in Blue
    x1,y1,x2,y2 = longest_right
    extra_right = LinearRegression().fit([[x1],[x2]],[[y1],[y2]])
    m_extta_right = extra_right.coef_[0]
    b_extta_right_ = extra_right.intercept_
    
    extra_right_x1 = (width*6) //10
    extra_right_x2 = width
    
    extra_right_y1 = int (m_extta_right*extra_right_x1 + b_extta_right_)
    extra_right_y2 = int (m_extta_right*extra_right_x2 + b_extta_right_)
    
    cv2.line(img, (extra_right_x1, extra_right_y1), (extra_right_x2, extra_right_y2), [0,0,255], 10)
 
    
    # Add extrapolated right to the linear regression
    X_right.append(extra_right_x1)
    Y_right.append(extra_right_y1)
    X_right.append(extra_right_x2)
    Y_right.append(extra_right_y2)
    
    
    
    
    #draw extrapolated longest left lines in Blue
    x1,y1,x2,y2 = longest_left
    extra_left = LinearRegression().fit([[x1],[x2]],[[y1],[y2]])
    m_extta_left = extra_left.coef_[0]
    b_extta_left_ = extra_left.intercept_
    
    extra_left_x1 = (width *2) //5
    extra_left_x2 = 0
    
    extra_left_y1 = int (m_extta_left*extra_left_x1 + b_extta_left_)
    extra_left_y2 = int (m_extta_left*extra_left_x2 + b_extta_left_)
    
    cv2.line(img, (extra_left_x1, extra_left_y1), (extra_left_x2, extra_left_y2), [0,0,255], 10)
 
    
    # Add extrapolated left to the linear regression
    X_left.append(extra_left_x1)
    Y_left.append(extra_left_y1)
    X_left.append(extra_left_x2)
    Y_left.append(extra_left_y2)
    
    
    
    #do a linear regression to get main right line
    X_right = np.reshape(X_right, (len(X_right),1))
    Y_right = np.reshape(Y_right, (len(Y_right),1))
    mdl = LinearRegression().fit(X_right,Y_right)
    m = mdl.coef_[0]
    b = mdl.intercept_
    
    
    #determine end points for the main right lane line
    # y = mx + b
    right_x1 = (width *3) //5
    right_x2 = width
    
    right_y1 = int (m*right_x1 + b)
    right_y2 = int (m*right_x2 + b) 
    
    #draw the main right lane line in White
    cv2.line(img, (right_x1, right_y1), (right_x2, right_y2), [255,255,255], 10)
    
          
    #do a linear regression to get main left line
    X_left = np.reshape(X_left, (len(X_left),1))
    Y_left = np.reshape(Y_left, (len(Y_left),1))
    mdl = LinearRegression().fit(X_left,Y_left)
    m = mdl.coef_[0]
    b = mdl.intercept_
    
    
    #determine end points for the main left lane line
    # y = mx + b
    left_x1 = (width *2) //5
    left_x2 = 0
    
    left_y1 = int (m*left_x1 + b)
    left_y2 = int (m*left_x2 + b) 
    
    #draw the main left lane line in White
    cv2.line(img, (left_x1, left_y1), (left_x2, left_y2), [255,255,255], 10)

    
    return img
            
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    filtered_line_img = draw_lines(line_img, lines)
    return filtered_line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [6]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

images = os.listdir("test_images/")

for img in images:
    #starting with reading image
    image = mpimg.imread("test_images/" + img)

    #Convert image to grayscale
    gray = grayscale(image)

 
    #Apply Guassian blur
    kernel_size = 5
    gray_blur = gaussian_blur(gray, kernel_size)



    #Canny edge detection
    low_threshold = 80
    high_threshold = 200
    edges = canny(gray_blur, low_threshold, high_threshold)


    #Apply region of interest mask
    hight = image.shape[0]
    width = image.shape[1]
    width_segment = int(width/2.5)  #width segment used in ROI vertics

    vertices = np.array([[(0,hight),(width_segment,320), ((width-width_segment),320), (width,hight)]], dtype=np.int32)
    masked_edges = region_of_interest(edges,vertices)

 
    #Run Hough transform
    rho = 2
    theta = np.pi / 180 
    threshold = 20
    min_line_len = 20 
    max_line_gap = 10


    lines_image = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)



    # Draw the lines on the initial image using weighted_img
    result_image = weighted_img(lines_image,image)
    plt.imshow(result_image)
    plt.figure()
      
    # write images with lines to test_images directory
    plt.savefig("test_images/" + "edited_" + img)
    
    



## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [7]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [8]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    #Convert image to grayscale
    gray = grayscale(image)

    #Apply Guassian blur
    kernel_size = 5
    gray_blur = gaussian_blur(gray, kernel_size)

    #Canny edge detection
    low_threshold = 80
    high_threshold = 200
    edges = canny(gray_blur, low_threshold, high_threshold)


    #Apply region of interest mask
    hight = image.shape[0]
    width = image.shape[1]
    width_segment = int(width/2.5)  #width segment used in ROI vertics

    vertices = np.array([[(0,hight),(width_segment,320), ((width-width_segment),320), (width,hight)]], dtype=np.int32)
    masked_edges = region_of_interest(edges,vertices)


    #Run Hough transform
    rho = 2
    theta = np.pi / 180 
    threshold = 20
    min_line_len = 20 
    max_line_gap = 10


    lines_image = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)


    #Draw the lines on the initial image using weighted_img
    result = weighted_img(lines_image,image)
    
    
    return result

Let's try the one with the solid white lane on the right first ...

In [9]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:04<00:00, 48.16it/s]     | 7/222 [00:00<00:03, 63.29it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 2.76 s, sys: 188 ms, total: 2.95 s
Wall time: 5.28 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [1]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

NameError: name 'HTML' is not defined

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [10]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:15<00:00, 44.08it/s]     | 6/682 [00:00<00:11, 58.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 8.98 s, sys: 504 ms, total: 9.49 s
Wall time: 16.2 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections


Assumptions:

- Finding lane assumes that there is at least one solid lane line for the left and the right.
- Works only on lines not curves.
- Assumes images are in clear weather
- Assumes the camera is stationary and at the same position.


The drawn lane lines not very stable, they jitters between frames, so we need an averaging
way among lane lines of the successive frames.

The idea of building the algorithm based on dominant lines on the left and write will not
work for curved roads.

Also the algorithm assumes at least there is one solid line one left and write will not make 
it valid for roads with dots marks instead of lines or roads with faded lines.




## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))